In [165]:
##The output file says for each cluster and pfam domain, the number of times it appears and the average Score it has
##In the end this script was not used but the average score might work as the likelihood of the pfam domain being present in the cluster

C:\Users\Yoo\Desktop\Summer Project\hmmer-3.2.1\trial


In [ ]:
cd C:\Users\Yoo\Desktop\Summer Project\hmmer-3.2.1\trial 

In [166]:
inputFile = open ("everycdspfamdomains.txt","r")
outputFile = open ("averageScorePerDomainPerCluster.txt","w")
clusterDictionary={}
currentCluster=""

In [167]:
##This code is kind of tricky. In the input file, the same cluster appears more than once (once for each CDS)
##So we have to know if we are still in the same cluster or if we are in a new one
##The lines that start with ">>" are the lines with the name of a domain
##Exactly three lines after that we find the score of the domain for that cluster
##However, there can be another score just below that one, so we have to turn on the flag "morePossibleScores" till we know for sure there are no more scores
##We know there is no more scores if the next line is empty. That is, the flag is on but "If Line:" returns false
##When the "lookingForTheScoreFlag" is 3, then we get the score as we said that the score is three lines after the domain name
##When checking the other lines with more possible scores, if the score is bigger we keep it, otherwise ignore it
##Each cluster finishes with "//" and we turn the newQueryFlag on
newQueryFlag = True
lookingForTheScoreFlag = 4
morePossibleScores=False
maxScore=0.0
for line in inputFile:
    line=line.strip("\n")
    if morePossibleScores:
        if line:
            #All those empty lines after the the possible scores just have a "\n" that has been remove two lines up
            #Only other lines with a score will be true then
            line=" ".join(line.split())
            line=line.split()
            if float(line[2]) > maxScore:
                maxScore=float(line[2])
        else:
            clusterDictionary[currentCluster][domainName][1]+=maxScore
            maxScore=0.0
            morePossibleScores=False
            lookingForTheScoreFlag=4
    elif newQueryFlag:
        if line[:6]=="Query:":
            line=" ".join(line.split())
            line=line.split("|")
            currentCluster=line[0][7:]
            if currentCluster not in clusterDictionary.keys():
                clusterDictionary[currentCluster]={}
            newQueryFlag = False
    else:
        if lookingForTheScoreFlag==4:
            if line[:2]=="//":
                newQueryFlag=True
            elif line[:3]==">> ":
                domainName=line[3:].split(" ")[0]
                if domainName not in clusterDictionary[currentCluster].keys():
                    clusterDictionary[currentCluster][domainName]=[1,0.0]
                else:
                    clusterDictionary[currentCluster][domainName][0] += 1
                lookingForTheScoreFlag=0
        else:
            lookingForTheScoreFlag +=1
            if lookingForTheScoreFlag==3:
                line=" ".join(line.split())
                line=line.split()
                maxScore=float(line[2])
                morePossibleScores=True

In [168]:
#Calculates the average of the Scores dividing the total score by the number of hits for one domain in the same cluster
for geneClusterName in clusterDictionary.keys():
    for domainName in sorted(clusterDictionary[geneClusterName].keys()):
        clusterDictionary[geneClusterName][domainName][1]= "%.2f" % (clusterDictionary[geneClusterName][domainName][1]/clusterDictionary[geneClusterName][domainName][0])

In [162]:
print (clusterDictionary)

{'BGC0000001': {'PCMT': [1, '89.40'], 'Methyltransf_31': [1, '38.20'], 'Methyltransf_25': [1, '33.20'], 'Methyltransf_3': [1, '19.90'], 'Methyltransf_23': [1, '20.10'], 'Methyltransf_11': [1, '18.60'], 'Fibrillarin': [1, '15.40'], 'CMAS': [1, '15.20'], 'ACP_syn_III_C': [2, '22.30'], 'Methyltransf_12': [1, '16.00'], 'GCD14': [1, '14.90'], 'MTS': [1, '13.30'], 'Methyltransf_24': [1, '14.00'], 'Methyltransf_32': [1, '13.20'], 'Methyltransf_18': [1, '13.00'], 'RrnaAD': [1, '11.40'], 'Ubie_methyltran': [1, '11.40'], 'Methyltransf_16': [1, '11.30'], 'BTAD': [2, '131.90'], 'Trans_reg_C': [2, '35.60'], 'TPR_10': [3, '10.33'], 'TPR_14': [2, '12.05'], 'p450': [2, '85.10'], 'GerE': [1, '55.50'], 'AAA_16': [2, '24.95'], 'TPR_2': [2, '16.75'], 'TPR_12': [2, '19.35'], 'TPR_19': [2, '17.70'], 'AAA_22': [2, '17.45'], 'Sigma70_r4_2': [1, '14.70'], 'HTH_10': [2, '11.50'], 'AAA_30': [2, '9.55'], 'TPR_7': [1, '20.40'], 'TPR_16': [1, '20.30'], 'TPR_1': [1, '19.90'], 'TPR_6': [1, '9.20'], 'TPR_8': [1, '12.6

In [169]:
##Writes to a file the whole cluster dictionary. Key is the cluster name and insideKey is the name of a domain
##For each domain we have [X, Y] where X is the number of times and Y is the average score
for key in clusterDictionary.keys():
    line=key+": "
    for insideKey in sorted(clusterDictionary[key].keys()):
        line+= insideKey + ": " + str(clusterDictionary[key][insideKey]) +"; "
    outputFile.write(line+"\n")

In [170]:
inputFile.close()
outputFile.close()